In [91]:
import requests
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


In [92]:
url = 'https://raw.githubusercontent.com/Lambda-School-Labs/tagger-ds/master/notebooks/merged_data_for_KNN.csv'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

15096914

In [93]:
labels = pd.read_csv("email.csv")

In [94]:
labels = labels.drop(columns = ['subject', 'content_type', 'to_who', 'from_who', 'dates',
                      'negative', 'positive', 'compound', 'final_pred', 'neutral',
                      'negative.1', 'neutral.1', 'positive.1', 'compound.1',
                      'final_pred.1', 'message_id', 'user_tags', 'thread_id' 
                      ])

In [95]:
labels

,smart_tags
0,['forecast']
1,"['travel', 'business', 'meeting', 'take', 'fun..."
2,"['test', 'successful', 'way']"
3,"['randy', 'send', 'schedule', 'salary', 'level..."
4,"['let', 'shoot', 'tuesday']"
...,...
9995,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ..."
9996,"['copy', 'idea', 'screw']"
9997,"[' ', 'forward', 'eric', 'basshouect', 'troy',..."
9998,"['buy', 'enron', 's']"


In [96]:
labeled_list = labels['smart_tags'].to_list()

In [97]:
labeled_list = [inner for item in labeled_list for inner in ast.literal_eval(item)] 

In [98]:
len(labeled_list)

1263664

In [99]:
labeled_list = set(labeled_list)

In [100]:
len(labeled_list)

45828

The following is a list of lists to be used as labels. labeled_list

In [101]:
len(labeled_list)

45828

Here is a list of search phrases. Each of these needs to be labeled by a KNN model with one of the labels above, ie. labeled_list

In [102]:
searches = ['trip to Austin', 'when is the first quarter report due', 'documentation for project', 'resolved issue with phillip', 'where are you on san marcos', 'vacation ideas']

In [103]:
url = 'https://raw.githubusercontent.com/Lambda-School-Labs/tagger-ds/master/notebooks/merged_data_for_KNN_with_body.csv'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

29340319

In [104]:
body = pd.read_csv("email.csv")

In [105]:
body

,email_body,subject,content_type,to_who,from_who,dates,negative,neutral,positive,compound,final_pred,smart_tags,negative.1,neutral.1,positive.1,compound.1,final_pred.1,message_id,user_tags,thread_id
0,here is our forecast,Subject:,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,['forecast'],0.000,1.000,0.000,0.0000,3.0,1,NaN,NaN
1,traveling to have a business meeting takes the...,Subject: Re:,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",0.000,0.893,0.107,0.9313,5,"['travel', 'business', 'meeting', 'take', 'fun...",0.000,0.893,0.107,0.9313,5.0,2,NaN,NaN
2,test successful way to go,Subject: Re: test,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",0.000,0.513,0.487,0.5859,4,"['test', 'successful', 'way']",0.000,0.513,0.487,0.5859,4.0,3,NaN,NaN
3,randy can you send me a schedule of the salary...,Subject:,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",0.000,1.000,0.000,0.0000,3,"['randy', 'send', 'schedule', 'salary', 'level...",0.000,1.000,0.000,0.0000,3.0,4,NaN,NaN
4,lets shoot for tuesday at,Subject: Re: Hello,Content-Type: text/plain; charset=us-ascii,NaN,From: phillip.allen@enron.com,"Date: Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",0.375,0.625,0.000,-0.3400,2,"['let', 'shoot', 'tuesday']",0.375,0.625,0.000,-0.3400,2.0,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,shes pretty sexy huh are we getting together t...,Subject: Re: Evite: Super Bowl Party,Content-Type: text/plain; charset=us-ascii,NaN,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:31:00 -0800 (PST)",0.031,0.799,0.170,0.9895,5,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ...",NaN,NaN,NaN,NaN,NaN,9996,NaN,NaN
9996,i copied your idea and it screwed up your name,Subject:,Content-Type: text/plain; charset=us-ascii,NaN,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 02:15:00 -0800 (PST)",0.262,0.738,0.000,-0.4939,2,"['copy', 'idea', 'screw']",NaN,NaN,NaN,NaN,NaN,9997,NaN,NaN
9997,forwarded by eric basshouect on am troy kluss...,Subject: 9912 Texas Financial Liquidations,Content-Type: text/plain; charset=us-ascii,NaN,From: eric.bass@enron.com,"Date: Tue, 25 Jan 2000 01:36:00 -0800 (PST)",0.000,0.914,0.086,0.4404,4,"[' ', 'forward', 'eric', 'basshouect', 'troy',...",NaN,NaN,NaN,NaN,NaN,9998,NaN,NaN
9998,did you buy any enron in the s,Subject:,Content-Type: text/plain; charset=us-ascii,NaN,From: eric.bass@enron.com,"Date: Wed, 6 Dec 2000 06:03:00 -0800 (PST)",0.000,1.000,0.000,0.0000,3,"['buy', 'enron', 's']",NaN,NaN,NaN,NaN,NaN,9999,NaN,NaN


In [106]:
body = body.drop(columns = [
'content_type',
'to_who',
'from_who',
'dates',
'negative',
'neutral',
'positive',
'compound',
'final_pred',
'negative.1',
'neutral.1',
'positive.1',
'compound.1',
'final_pred.1',
'message_id',
'user_tags',
'thread_id'])


In [107]:
body = body.dropna()

In [108]:
body

,email_body,subject,smart_tags
0,here is our forecast,Subject:,['forecast']
1,traveling to have a business meeting takes the...,Subject: Re:,"['travel', 'business', 'meeting', 'take', 'fun..."
2,test successful way to go,Subject: Re: test,"['test', 'successful', 'way']"
3,randy can you send me a schedule of the salary...,Subject:,"['randy', 'send', 'schedule', 'salary', 'level..."
4,lets shoot for tuesday at,Subject: Re: Hello,"['let', 'shoot', 'tuesday']"
...,...,...,...
9995,shes pretty sexy huh are we getting together t...,Subject: Re: Evite: Super Bowl Party,"['s', 'pretty', 'sexy', 'huh', 'get', 'talk', ..."
9996,i copied your idea and it screwed up your name,Subject:,"['copy', 'idea', 'screw']"
9997,forwarded by eric basshouect on am troy kluss...,Subject: 9912 Texas Financial Liquidations,"[' ', 'forward', 'eric', 'basshouect', 'troy',..."
9998,did you buy any enron in the s,Subject:,"['buy', 'enron', 's']"


In [133]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(body)
y = vectorizer.fit_transform(searches)
#kmeans = KMeans(n_clusters=2).fit(X)

In [134]:
modelkmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, n_init=100)
modelkmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=200,
       n_clusters=3, n_init=100, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [135]:
modelkmeans.cluster_centers_

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [136]:
modelkmeans.labels_

array([0, 1, 2], dtype=int32)

In [141]:
predicted_labels_kmeans = modelkmeans.predict(X)

In [143]:
print("\n",searches[0],":",predicted_labels_kmeans[0],\
        "\n",searches[1],":",predicted_labels_kmeans[1],\
        "\n",searches[2],":",predicted_labels_kmeans[2])


 trip to Austin : 0 
 when is the first quarter report due : 1 
 documentation for project : 2
